In [1]:
from sklearn.feature_extraction import DictVectorizer
import xgboost as xgb
import pandas as pd
import numpy as np
import pickle
import os
import re
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
from hdfs import InsecureClient
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from collections import OrderedDict
client_hdfs = InsecureClient("http://10.65.31.100:50070")

In [2]:
def data_read(url):
    with client_hdfs.read(url , encoding = 'utf-8') as reader:
        if client_hdfs.content(url)["spaceConsumed"]>1000:
            df_input = pd.read_csv(reader,names=["name"])
            return df_input

In [3]:
with client_hdfs.read( "/user/dataplatform/indicium/temp/NM/newregistration12345/part-00000" , encoding = 'utf-8') as reader:
    df = pd.read_csv(reader)   

In [4]:
df.head()

,imsId,guid,displayName,firstName,middleName,lastName,gender
0,VjAxI2E0M2FjNmJmLTQ2MTAtNDY3Ny1iMDkzLTRlZTQzOD...,d0a75161-edfd-4348-a1d2-b41a66ae287f,kumar bhoir,kumar,NaN,bhoir,m
1,VjAxI2E0M2JkODQ3LTYzN2YtNDc5My1hOGMwLWZiNDgwMT...,645ffcf7-abcc-4c6b-93d2-7af6fae5c1be,shreyash gangwar,shreyash,NaN,gangwar,m
2,VjAxI2E0M2NlMzcwLWQ3YWItNGU0Mi05YTE5LTZlOTkyNT...,1847122d-9dce-497c-a12b-eaf7c475c902,juned,juned,NaN,NaN,m
3,VjAxI2E0M2Q3NTU0LTVhZTQtNDJmYS04MmE1LWJhYTk0YW...,2142d281-77fa-4021-aeb5-b2b241c15951,kranthi kumar,kranthi,NaN,kumar,m
4,VjAxI2E0MDAyNDM1LTQ1NzktNGI5ZC05YzFlLWQ2NzFkMj...,6c4901e0-9b45-4d9e-ba5c-c482a426b3e9,ramesh chariya,ramesh,NaN,chariya,f


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486164 entries, 0 to 23486163
Data columns (total 7 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   imsId        object
 1   guid         object
 2   displayName  object
 3   firstName    object
 4   middleName   object
 5   lastName     object
 6   gender       object
dtypes: object(7)
memory usage: 1.2+ GB


In [6]:
df=df[df["guid"].notna()]

In [7]:
df.fillna(' ', inplace=True)

In [8]:
imsMaping = df[["guid","imsId"]].reset_index(drop=True)
display(imsMaping.shape)
imsMaping = imsMaping.drop_duplicates(subset={'guid'})
display(imsMaping.shape)

(23486164, 2)

(23151963, 2)

In [9]:
df.drop_duplicates(subset={'guid','displayName','firstName','middleName','lastName','gender'},inplace=True)
df = df.replace({'default':' '})
df['FullName'] = df['displayName']+ " " + df['firstName'] + " " + df['middleName'] + " " +df['lastName']

In [10]:
def remove_punctuations(text):
    rmvSpcchar = re.compile("[/(){}\[\]\|&@,;_'.-]")
    rmvBadSym = re.compile('[^\d+\0-9a-z#+_]')
    rmvNumbers = re.compile('\d+')
    text = re.sub(rmvSpcchar, ' ', text)
    text = re.sub(rmvBadSym, ' ', text)
    text = re.sub(rmvNumbers, ' ', text)
    return text

In [11]:
df["FullNameCleaned"] = df["FullName"].apply(lambda x: remove_punctuations(x))

In [12]:
df['Cleaned Name']=df['FullNameCleaned'].apply(lambda x: list(filter(None,OrderedDict.fromkeys(x.split(" ")))))

In [13]:
df['Cleaned Name']

0                [kumar, bhoir]
1           [shreyash, gangwar]
2                       [juned]
3              [kranthi, kumar]
4             [ramesh, chariya]
                   ...         
23486159       [karthik, gumma]
23486160         [guri, kainth]
23486161      [sadiq, siddiuqe]
23486162        [akash, dhurwe]
23486163    [day, aram, bairwa]
Name: Cleaned Name, Length: 23277334, dtype: object

In [14]:
df.drop(['firstName','middleName','lastName'],axis=1,inplace=True)

In [15]:
df['Counts'] = df['Cleaned Name'].apply(lambda x: len(x))
index_names = df[ df["Counts"]==0 ].index
df.drop(index_names, inplace = True)

In [16]:
df['FirstName'] = df['Cleaned Name'].apply(lambda x: x[0])

In [17]:
df["FirstNameLength"] = df["FirstName"].apply(lambda x: len(x))
df = df[df["FirstNameLength"]>=3]

In [18]:
df['LastName'] = df['Cleaned Name'].apply(lambda x: x[1] if len(x)>1 else "")

In [19]:
df['ExtendedName'] = df['Cleaned Name'].apply(lambda x: " ".join(x[2:]) if len(x)>2 else "")

In [20]:
df[df['FirstName']=='unknown']

,imsId,guid,displayName,gender,FullName,FullNameCleaned,Cleaned Name,Counts,FirstName,FirstNameLength,LastName,ExtendedName
11826,VjAxI2VlMWJhMzI5LTEzNzEtNDcxNi04OTNhLTg4MmQzNz...,502087a9-a9fb-476f-936e-d2de4c5d8496,unknown user,f,unknown user unknown user,unknown user unknown user,"[unknown, user]",2,unknown,7,user,
72190,VjUwIzcyZGM3ODgzLTg4ZTItNDkxZS04YWI4LThjYjQzNj...,c65c45b2-1bee-47e4-abda-547c2ed8c98b,unknown vaibhav,m,unknown vaibhav unknown vaibhav,unknown vaibhav unknown vaibhav,"[unknown, vaibhav]",2,unknown,7,vaibhav,
121338,VjUwI2JiZjZhNDZjLTE0MDgtNDdkOS1hM2I3LWZkYmVlNm...,978bf437-ebfc-495b-a1b4-f23bbf923e2d,unknown gaming official,m,unknown gaming official unknown gaming officia...,unknown gaming official unknown gaming officia...,"[unknown, gaming, official]",3,unknown,7,gaming,official
122635,VjUwI2M4NTQzMTBjLTgwODAtNDY4MC05NmFmLWM5ZjU4Y2...,3061efc8-2760-4870-aa12-d67881c0eeb9,unknown gaming galaxie,m,unknown gaming galaxie unknown gaming galaxie ...,unknown gaming galaxie unknown gaming galaxie ...,"[unknown, gaming, galaxie]",3,unknown,7,gaming,galaxie
146803,VjUwIzY1NDI5YjVmLWI3MGQtNGNiMC1hNDFjLTkwNjcxOW...,44fc10f6-3a38-4fe6-89b3-1cfa63ae3ac9,unknown name,m,unknown name unknown name name,unknown name unknown name name,"[unknown, name]",2,unknown,7,name,
...,...,...,...,...,...,...,...,...,...,...,...,...
23235768,VjUwIzJjZWVjMjdkLTRiMDQtNDA5OC05ZDI0LTg3MmIwYW...,2433cd18-e66b-4e54-89e6-73ace3d46b80,unknown boy juyel,m,unknown boy juyel unknown boy juyel juyel,unknown boy juyel unknown boy juyel juyel,"[unknown, boy, juyel]",3,unknown,7,boy,juyel
23239477,VjUwIzQ5MmQ0NzQzLTBkMjQtNDY2NC1hNGZkLTkyNGNhND...,1d6e6066-7aed-4ec8-9dd3-361695dcfbd6,unknown brand,m,unknown brand unknown brand,unknown brand unknown brand,"[unknown, brand]",2,unknown,7,brand,
23296677,VjUwI2M0MzM1MWVjLWU3YjctNGQ1OS05OWFjLTVjYjYzMm...,73784003-ccfb-4795-9d72-f346f1bbc5ae,unknown,m,unknown unknown,unknown unknown,[unknown],1,unknown,7,,
23346999,VjAxIzBhMjU1OGJlLTFkZTUtNDhhOC1iMDNkLTc2ZGJjMm...,d259d738-413e-4621-a663-90f595464985,unknown,m,unknown unknown gonugunta,unknown unknown gonugunta,"[unknown, gonugunta]",2,unknown,7,gonugunta,


In [21]:
directRecommendation = df[df["FirstName"].isin(['mr','mrs','smt','kumar','kumari','miss','shri','mohd'])]
df.drop(directRecommendation.index,inplace=True)

In [22]:
toRemove = df[df["FirstName"].isin(['dr','md','all','gaming','technical','royal','the','day','unknown'])]
df.drop(toRemove.index,inplace=True)
df = df.reset_index(drop=True)

In [23]:
df[df['FirstName']=='unknown']

,imsId,guid,displayName,gender,FullName,FullNameCleaned,Cleaned Name,Counts,FirstName,FirstNameLength,LastName,ExtendedName


In [24]:
df["FirstName"].value_counts()

rahul               194658
amit                132530
deepak              124740
ravi                115660
ajay                106969
                     ...  
samiton                  1
thyageshwar              1
maghbara                 1
prasenjitdebnath         1
adithay                  1
Name: FirstName, Length: 1001241, dtype: int64

In [25]:
FnFreq = df["FirstName"].value_counts().reset_index()
fnList = FnFreq["index"].tolist()

In [26]:
LnFreq = df["LastName"].value_counts().reset_index()
lnList = LnFreq["index"].tolist()

In [27]:
nameMerge = FnFreq.merge(LnFreq, on ="index", how="inner")

In [28]:
finalDf = df[["imsId","guid","gender","FirstName"]].reset_index(drop=True)

In [29]:
def is_vowel(char):

    if char in ('a', 'e', 'i', 'o', 'u'):
        return'1'
    else:
        return'0' 

In [30]:
finalDf["Vowel_Present"] = finalDf["FirstName"].apply(lambda x: is_vowel(x[-1]))

In [31]:
gender_count=finalDf[['gender','Vowel_Present']].value_counts()

In [32]:
gender_count

gender  Vowel_Present
m       0                13980989
        1                 5388856
f       1                 1863700
        0                 1262595
o       0                    7163
        1                    3999
dtype: int64

In [33]:
Females = finalDf[finalDf["gender"]=="f"]
Males = finalDf[finalDf["gender"]=="m"]

In [34]:
Females_0=Females[Females['Vowel_Present']=='0'] #no vowel female gender
Females_0

,imsId,guid,gender,FirstName,Vowel_Present
3,VjAxI2E0MDAyNDM1LTQ1NzktNGI5ZC05YzFlLWQ2NzFkMj...,6c4901e0-9b45-4d9e-ba5c-c482a426b3e9,f,ramesh,0
32,VjAxI2E0MzY4YWQyLWJhNTItNDVkYy1hOGNjLWJhNzFiNz...,df0b6848-2880-4957-88e5-183051d76e8d,f,thanush,0
38,VjAxI2E0NDQ4MDgyLWYyMDAtNGJhZC05MWFlLWQyY2FjOD...,6b0d20ef-f95f-4ded-82c7-2df91af0318c,f,sagar,0
54,VjAxI2E0NjM2NGY2LWZmNWEtNDgxOC1hOTIzLTAyNzQxMT...,ad2e253f-3e1b-450a-950f-b2a5588c8f5a,f,shekh,0
56,VjAxI2E0NjRlNThiLWY2MDAtNDAwOS1iY2ZlLWJjNTcwYT...,db9ac3ac-f15e-44ec-8394-c06dd24f1476,f,rangrej,0
...,...,...,...,...,...
22506545,VjUwIzliNDc4MGM0LWRhZDEtNDkzOS04M2IzLTUxZmUwOD...,28b2e97e-3e90-4af4-ae7b-c449a6c8d1da,f,anees,0
22506903,VjUwIzlkYTk3MzBhLTY1MGEtNGJjZC1hMmM3LTAxNzFjZD...,82fb67a6-8fed-4465-9a4e-a9b086d6bf05,f,meher,0
22506981,VjUwIzllMWUzNzU1LWViYjEtNDIyMi1hYmEzLTQ1NWJhM2...,36124e94-6858-4c65-be65-89907012bc7b,f,suman,0
22507209,VjUwIzlmODFhOWYxLWRlOGItNDAzZi05MDM2LTlmM2FkYm...,967c1919-f7cc-48f1-89fb-36067b42fbcf,f,jasmeet,0


In [35]:
Males_1=Males[Males['Vowel_Present']=='1']  #with vowel male gender
Males_1

,imsId,guid,gender,FirstName,Vowel_Present
2,VjAxI2E0M2Q3NTU0LTVhZTQtNDJmYS04MmE1LWJhYTk0YW...,2142d281-77fa-4021-aeb5-b2b241c15951,m,kranthi,1
4,VjAxI2E0MDE1Y2U3LTJjYmQtNGJmNy1iNDRjLWRiODY0N2...,a1fc1522-4921-4571-9638-ab6d083382a1,m,satti,1
7,VjAxI2E0MDQ5YmIxLTg4ZWItNDBmMS04ZjA1LWViMDg0NW...,fdc23caf-94ce-4fca-94ab-4cf9a8254c49,m,kandiri,1
20,VjAxI2E0MjA5Yjg0LTVmNWItNDhjZC1hZWJjLTA4N2EzMz...,8d85a7cb-feaf-4a6b-9d7d-d9bb3b7cb44d,m,devendra,1
27,VjAxI2E0MzFhNmMyLTgyNjYtNDEzYS04YzRmLTM1ZjY2Mz...,da6b7cda-07e5-453a-9ba1-4c7e6627cccc,m,ravindra,1
...,...,...,...,...,...
22507285,VjUwIzlmZWU3NWVkLTI2OWYtNDg4OC1iYWE1LWU4YWJhMz...,46ff84cf-fb8f-4ebf-a201-fb9db4d417fc,m,santu,1
22507286,VjUwIzlmZWY0NGVjLWQ5NzctNDhhYi05M2RiLTRjZjc4YT...,95b56977-89f6-48c7-afb8-ee697a883882,m,rama,1
22507288,VjUwIzlmZWZkNjllLTU0OTItNGRkOS05Zjk4LWJlMzNjOW...,460a4d8d-95fe-4ce3-99c1-7dd39444564b,m,saloni,1
22507294,VjUwIzlmZjY2NThlLThiZTQtNDgzNS1iZThmLTZiYWE2NT...,419c81dd-ca8b-404a-b2a4-5888edef7c91,m,pasunuri,1


In [36]:
df['gender'].value_counts()

m    19369845
f     3126295
o       11162
Name: gender, dtype: int64

In [37]:
def features(name):
    name=str(name)
    name = (name.lower()).split()[0]
    return {
        'first-letter': name[0], # First letter
        'first2-letters': name[0:2], # First 2 letters
        'first3-letters': name[0:3], # First 3 letters
        'first4-letters': name[0:4],
        'last-letter': name[-1],
        'vowel': is_vowel(name[-1]),
        'last2-letters': name[-2:],
        'last3-letters': name[-3:],
    }

In [38]:
finalDf['binaryGender'] = finalDf['gender'].replace({'m':0,'f':1})

In [39]:
X=finalDf.drop(['gender','binaryGender','Vowel_Present'],axis=1)

In [40]:
y=finalDf['binaryGender']

In [41]:
#y=y.astype(str)

In [42]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.25, random_state=42)

In [43]:
features = np.vectorize(features)

In [44]:
#X_train=X_train.astype('str')

In [45]:
X_train["flag"] = X_train["FirstName"].apply(lambda x: x.isalpha())

/home/datascience/yes/envs/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
test = X_train[X_train["flag"]==False]

In [ ]:
test

In [47]:
namesData=features(X_train['FirstName'])

In [48]:
vectorizer = DictVectorizer()

In [ ]:
vect = vectorizer.fit_transform(namesData)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs',max_iter=1000)

In [ ]:
#X_train.shape

In [ ]:
#clf.fit(vect, y_train) 

In [ ]:
result=clf.predict(vectorizer.transform((X_test)))

In [ ]:
result

In [ ]:
y_test['prediction'] = result

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
#df_pred=pd.merge(finalDff,y_test[['prediction']],how = 'left',left_index = True, right_index = True)

In [ ]:
#print('Accuracy of logistic regression'.format(clf.score(X_test, y_test)))